# Cynthia Kielpinski

## Research Interests:

By majoring in statistics, I have always had a passion for the process behind statistics. The collection of data, the wrangling and processing of the data, exploratory analysis of the data, interpreting the analysis, and then finalizing the process by formulating a conclusion from the analysis. One fundamental ideal in statistics I have learned is that without an interdiscplinary field, you do not have the data and you are simply performing pure statistics. As per this comment, I have never found interest in pure statistics thus I found a passion in the interdisciplinary field of ecology and environmental sciences. For this project, my research interests are focus on exploratory statistics (required) and an environmental or ecological factor. Environmental data is more prominent  than ecological from my persepctive thus I have decided to focus on an environmental influence. Since our group decided on usng a summer and winter Olympics dataset, I though to refine my first research question to involve different countries performance in the summer and winter olympics influenced by their "local" climate. Competition is always a prfound topic, and I was also interested in how countries perform in their home country versus a visiting country. 

## Research Question: (updated 03-01-2023)

1. **Does the annual average temperature of a country determine a correlation between the probability or likelihood of preforming more successfully in the summer or winter Olympics in individual disciplines?** 

I have decided to use a Bernoulli distribution as the designation for "hot" (1) and "cold" (0) climates. I used the annual temperature column to calculate the overall average annual temperature for all the countries in the dataset to determine a "threshold" for determining if a country is "hot" and "cold" by using a Bernoulli distribution. For example, does Canada have a better probability of winning medals in the winter Olympics since it is considered to be a cold climate according to my Bernoulli distribution? Does Brazil have a better probability of winning medals in the summer Olympics since it is in the hot designation? Or is there no correlation between a countries climate classification and which Olympics the country preforms better in? I will explore the correlations between countries and their climates in regards to number of medals won in each of the respective summer or winter Olympic events. Overall, to answer this research question, my main form of analysis will be a multiple linear regression using categorical(dummy) variables and quantitative variables.

## Exploratory Data Analysis Plan: (updated 03-01-2023)

To determine some answers into starting to answer my research question, I will use descriptive and exploratory techniques such as... Some of my ideas for more advanced analyses are: multiple linear regression analysis, and potentially using principal component analysis. From briefly looking over the datasets, the above methods are a brief thought to the possibilities when exploring the data. After wrangling the data, I will define the specific analyses methods I choose to use. I have a background in Statistics, thus deciding what analyses will be used at a later stage will be an simplistic decision. 

I will complete as little wrangling as needed, but I have visioned the data to be cleaner and more orderly without taking hours on wrangining. I think my current plan is to edit/wrangle each dataset, and then merge all of the separate datasets together so I have one uniform dataset to work with. I will change some column names for ease of analyses, and try to keep everything simplistic. Some columns I am thinking of adding as of now are: total number of medals won, total number of gold, silver and bronze medals won, and maybe dome column for a way to rank the total number of "points" won to easily compute standings.

## Import Packages: (all packages needed for analysis, updated 02-12-2023)

In [53]:
## Import Packages:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
## Data Upload:
OlympicHost = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_hosts.csv")

OlympicMedals = pd.read_csv("../../data/raw/Olympic_Summer_Winter_Games_1986-2022/olympic_medals.csv")

GDPCountry = pd.read_csv("../../data/raw/GDPbyCountry.csv")

HistTemp = pd.read_csv("../../data/raw/HistoricalTemperatureData.csv")


#### Notes: About the Historical Temperature Data and the GDP Datasets

The following links are from the World Bank Website which has the following license as described below. 

[Historical Average Temperature Data per Country](https://datacatalog.worldbank.org/search/dataset/00402760) 

[Gross Domestic Product Data:](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD)

By looking into the licenses for the data from the [World Bank Website](https://www.worldbank.org/en/about/legal/terms-of-use-for-datasets) it quotes that the license "[u]nless specifically labeled otherwise, these Datasets are provided to you under a Creative Commons Attribution 4.0 International License (CC BY 4.0), with the additional terms below."


## Initial Edits and Observations: 

In [55]:
## Primary Use:
## Looking at the summer versus winter olympics here, need game_slug to merge

OlympicHost = OlympicHost.drop(columns=["game_end_date", "game_start_date"])
OlympicHost.head()

,game_slug,game_location,game_name,game_season,game_year
0,beijing-2022,China,Beijing 2022,Winter,2022
1,tokyo-2020,Japan,Tokyo 2020,Summer,2020
2,pyeongchang-2018,Republic of Korea,PyeongChang 2018,Winter,2018
3,rio-2016,Brazil,Rio 2016,Summer,2016
4,sochi-2014,Russian Federation,Sochi 2014,Winter,2014


In [66]:
OlympicMedals = OlympicMedals.rename(columns={"discipline_title": "discipline", "slug_game": "game_slug","event_title": "event","event_gender": "event_sex",
                             "medal_type": "medal_type","participant_type": "participant_type","participant_title": "participant_country",
                              "country_name": "country","country_3_letter_code": "country_code"})

OlympicMedals = OlympicMedals[OlympicMedals["participant_type"] == "Athlete"]

## Establish Point Values:
point_value = []
for row in OlympicMedals["medal_type"]:
        if row == "GOLD" :    point_value.append(3)
        elif row == "SILVER" :  point_value.append(2)
        else:           point_value.append(1)
        
OlympicMedals["point_value"] = point_value

##OlympicMedals = OlympicMedals.drop(columns=["athlete_url", "country_code", "athlete_full_name"])
OlympicMedals = OlympicMedals.drop(columns=["participant_country"])
OlympicMedals.head(15)



,discipline,game_slug,event,event_sex,medal_type,participant_type,country,country_code,point_value
12,Freestyle Skiing,beijing-2022,Men's Moguls,Men,SILVER,Athlete,Canada,CAN,2
13,Freestyle Skiing,beijing-2022,Men's Moguls,Men,GOLD,Athlete,Sweden,SWE,3
14,Freestyle Skiing,beijing-2022,Men's Moguls,Men,BRONZE,Athlete,Japan,JPN,1
15,Freestyle Skiing,beijing-2022,Men's Freeski Halfpipe,Men,GOLD,Athlete,New Zealand,NZL,3
16,Freestyle Skiing,beijing-2022,Men's Freeski Halfpipe,Men,SILVER,Athlete,United States of America,USA,2
17,Freestyle Skiing,beijing-2022,Men's Freeski Halfpipe,Men,BRONZE,Athlete,United States of America,USA,1
18,Freestyle Skiing,beijing-2022,Men's Freeski Big Air,Men,BRONZE,Athlete,Sweden,SWE,1
19,Freestyle Skiing,beijing-2022,Men's Freeski Big Air,Men,GOLD,Athlete,Norway,NOR,3
20,Freestyle Skiing,beijing-2022,Men's Freeski Big Air,Men,SILVER,Athlete,United States of America,USA,2
21,Freestyle Skiing,beijing-2022,Men's Ski Cross,Men,BRONZE,Athlete,ROC,ROC,1


In [37]:
HistTemp = HistTemp.rename(columns = {"ISO_3DIGIT": "country_code", "Jan_Temp": "Jan_temp", 
                                      "Apr_Temp": "Apr_temp", "Jun_Temp": "Jun_temp", "July_Temp":"July_temp",
                                      "Aug_Temp": "Aug_temp", "Nov_Temp": "Nov_temp"})

HistTemp1 = HistTemp.drop(columns=["Jan_temp", "Feb_temp", "Mar_temp", "Apr_temp", "May_temp", "Jun_temp",
                                 "July_temp", "Aug_temp", "Sept_temp", "Oct_temp", "Nov_temp", "Dec_temp"])

## Mean Temp: 17.965506 degrees celsius
MeanTemp = HistTemp1[["Annual_temp"]].mean()

## Dummy Variable: Categorical (1: Hot, 0: Cold)
hot_cold = []
for row in HistTemp1["Annual_temp"]:
        if row > 17.95 :    hot_cold.append(1)
        else:           hot_cold.append(0)
        
HistTemp1["hot_cold"] = hot_cold
HistTemp1.head(10)

,country_code,Annual_temp,hot_cold
0,AFG,12.92,0
1,AGO,21.51,1
2,ALB,11.27,0
3,ARE,26.83,1
4,ARG,14.22,0
5,ARM,6.37,0
6,AUS,21.51,1
7,AUT,6.19,0
8,AZE,12.22,0
9,BDI,20.27,1


In [82]:
## This data set is going to need a bit of wrangling... (HUGE YIKES -_-)
## GDPCountry = GDPCountry.drop([0,2])
## GDPCountry = GDPCountry.drop([1])
## GDPCountry = GDPCountry.drop(columns=["Unnamed: 3"])
## GDPCountry = GDPCountry.drop([3])
GDPCountry = GDPCountry.rename(columns = {"Data Source": "country_name", "World Development Indicators": "country_code", 
                                          "Unnamed: 2": "GDP", "Unnamed: 4": "1960", "Unnamed: 5": "1961", "Unnamed: 6": "1962",
                                          "Unnamed: 7": "1963", "Unnamed: 8": "1964", "Unnamed: 9": "1965", "Unnamed: 10": "1966",
                                          "Unnamed: 11": "1967", "Unnamed: 12": "1968", "Unnamed: 13": "1969", "Unnamed: 14": "1970",
                                          "Unnamed: 15": "1971","Unnamed: 16": "1972","Unnamed: 17": "1973","Unnamed: 18": "1974",
                                          "Unnamed: 19": "1975","Unnamed: 20": "1976","Unnamed: 21": "1977","Unnamed: 22": "1978",
                                          "Unnamed: 23": "1979","Unnamed: 24": "1980","Unnamed: 25": "1981","Unnamed: 26": "1982",
                                          "Unnamed: 27": "1983","Unnamed: 28": "1984","Unnamed: 29": "1985","Unnamed: 30": "1986",
                                          "Unnamed: 31": "1987","Unnamed: 32": "1988","Unnamed: 33": "1989","Unnamed: 34": "1990",
                                          "Unnamed: 35": "1991","Unnamed: 36": "1992","Unnamed: 37": "1993","Unnamed: 38": "1994",
                                          "Unnamed: 39": "1995","Unnamed: 40": "1996","Unnamed: 41": "1997","Unnamed: 42": "1998",
                                          "Unnamed: 43": "1999","Unnamed: 44": "2000","Unnamed: 45": "2001","Unnamed: 46": "2002",
                                          "Unnamed: 47": "2003","Unnamed: 48": "2004","Unnamed: 49": "2005","Unnamed: 50": "2006",
                                          "Unnamed: 51": "2007","Unnamed: 52": "2008","Unnamed: 53": "2009","Unnamed: 54": "2010",
                                          "Unnamed: 55": "2011","Unnamed: 56": "2012","Unnamed: 57": "2013","Unnamed: 58": "2014",
                                          "Unnamed: 59": "2015","Unnamed: 60": "2016","Unnamed: 61": "2017","Unnamed: 62": "2018",
                                          "Unnamed: 63": "2019","Unnamed: 64": "2020","Unnamed: 65": "2021"})

GDPCountry.to_csv("../../data/raw/New_GDP.csv")

In [67]:
## Merge Data Sets:

## Exploratory Data Analysis: 